<a href="https://colab.research.google.com/github/ffer200395/ML-course/blob/main/5_PySpark_Rese%C3%B1as.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Instalamos el framework pySpark
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.4 MB 39 kB/s 
     |████████████████████████████████| 199 kB 29.0 MB/s 
  Created wheel for pyspark: filename=pyspark-3.3.1-py2.py3-none-any.whl size=281845512 sha256=924140c72e4e98f0a691aca5213e65e83796a88c4fb189f97919fb062e9c3d29
  Stored in directory: /root/.cache/pip/wheels/43/dc/11/ec201cd671da62fa9c5cc77078235e40722170ceba231d7598
Successfully built pyspark


In [ ]:
from pyspark.sql import SparkSession
from google.colab import drive
import pandas as pd

In [ ]:
spark = SparkSession.builder.appName("Python Spark Reseñas").getOrCreate()

In [ ]:
# Path
path = '/content/gdrive/MyDrive/Colab Notebooks/Formación Python y ML'
# Mount drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
df_spark = spark.read.options(delimiter=",", header=True,escape="\"").csv(path+'/IMDB Dataset.csv')
df_spark.printSchema()
df_spark.show()

root
 |-- review: string (nullable = true)
 |-- sentiment: string (nullable = true)

+--------------------+---------+
|              review|sentiment|
+--------------------+---------+
|One of the other ...| positive|
|A wonderful littl...| positive|
|I thought this wa...| positive|
|Basically there's...| negative|
|Petter Mattei's "...| positive|
|Probably my all-t...| positive|
|I sure would like...| positive|
|This show was an ...| negative|
|Encouraged by the...| negative|
|If you like origi...| positive|
|Phil the Alien is...| negative|
|I saw this movie ...| negative|
|So im not a big f...| negative|
|The cast played S...| negative|
|This a fantastic ...| positive|
|Kind of drawn in ...| negative|
|Some films just s...| positive|
|This movie made i...| negative|
|I remember this f...| positive|
|An awful film! It...| negative|
+--------------------+---------+
only showing top 20 rows



In [ ]:
# Convertimos la variable respuesta en formato numérico 
from pyspark.ml.feature import StringIndexer
indexer = StringIndexer(inputCol="sentiment", outputCol="sentiment_enc") 
indexed = indexer.fit(df_spark).transform(df_spark) 
indexed.show()

+--------------------+---------+-------------+
|              review|sentiment|sentiment_enc|
+--------------------+---------+-------------+
|One of the other ...| positive|          1.0|
|A wonderful littl...| positive|          1.0|
|I thought this wa...| positive|          1.0|
|Basically there's...| negative|          0.0|
|Petter Mattei's "...| positive|          1.0|
|Probably my all-t...| positive|          1.0|
|I sure would like...| positive|          1.0|
|This show was an ...| negative|          0.0|
|Encouraged by the...| negative|          0.0|
|If you like origi...| positive|          1.0|
|Phil the Alien is...| negative|          0.0|
|I saw this movie ...| negative|          0.0|
|So im not a big f...| negative|          0.0|
|The cast played S...| negative|          0.0|
|This a fantastic ...| positive|          1.0|
|Kind of drawn in ...| negative|          0.0|
|Some films just s...| positive|          1.0|
|This movie made i...| negative|          0.0|
|I remember t

In [ ]:
# Ingeniería de características
from pyspark.ml.feature import RegexTokenizer, StopWordsRemover, CountVectorizer, IDF

# Dejamos solo palabras
regexTokenizer = RegexTokenizer(inputCol="review", outputCol="tokens", pattern="\\W", toLowercase = True, minTokenLength = 3)
df_tokens = regexTokenizer.transform(indexed)
df_tokens.show()

+--------------------+---------+-------------+--------------------+
|              review|sentiment|sentiment_enc|              tokens|
+--------------------+---------+-------------+--------------------+
|One of the other ...| positive|          1.0|[one, the, other,...|
|A wonderful littl...| positive|          1.0|[wonderful, littl...|
|I thought this wa...| positive|          1.0|[thought, this, w...|
|Basically there's...| negative|          0.0|[basically, there...|
|Petter Mattei's "...| positive|          1.0|[petter, mattei, ...|
|Probably my all-t...| positive|          1.0|[probably, all, t...|
|I sure would like...| positive|          1.0|[sure, would, lik...|
|This show was an ...| negative|          0.0|[this, show, was,...|
|Encouraged by the...| negative|          0.0|[encouraged, the,...|
|If you like origi...| positive|          1.0|[you, like, origi...|
|Phil the Alien is...| negative|          0.0|[phil, the, alien...|
|I saw this movie ...| negative|          0.0|[s

In [ ]:
stopwords_remover = StopWordsRemover(inputCol='tokens',outputCol='filtered_tokens',stopWords=['this','these','that','those','the','and'])
df_sw = stopwords_remover.transform(df_tokens)
df_sw.show()

+--------------------+---------+-------------+--------------------+--------------------+
|              review|sentiment|sentiment_enc|              tokens|     filtered_tokens|
+--------------------+---------+-------------+--------------------+--------------------+
|One of the other ...| positive|          1.0|[one, the, other,...|[one, other, revi...|
|A wonderful littl...| positive|          1.0|[wonderful, littl...|[wonderful, littl...|
|I thought this wa...| positive|          1.0|[thought, this, w...|[thought, was, wo...|
|Basically there's...| negative|          0.0|[basically, there...|[basically, there...|
|Petter Mattei's "...| positive|          1.0|[petter, mattei, ...|[petter, mattei, ...|
|Probably my all-t...| positive|          1.0|[probably, all, t...|[probably, all, t...|
|I sure would like...| positive|          1.0|[sure, would, lik...|[sure, would, lik...|
|This show was an ...| negative|          0.0|[this, show, was,...|[show, was, amazi...|
|Encouraged by the...

In [ ]:
# Realizamos un conteo de las palabras en cada review
vectorizer = CountVectorizer(inputCol='filtered_tokens',outputCol='rawFeatures',maxDF=.8,minDF=.05,binary=True)
df_vect = vectorizer.fit(df_sw).transform(df_sw)
df_vect.show()

+--------------------+---------+-------------+--------------------+--------------------+--------------------+
|              review|sentiment|sentiment_enc|              tokens|     filtered_tokens|         rawFeatures|
+--------------------+---------+-------------+--------------------+--------------------+--------------------+
|One of the other ...| positive|          1.0|[one, the, other,...|[one, other, revi...|(390,[0,2,3,4,6,7...|
|A wonderful littl...| positive|          1.0|[wonderful, littl...|[wonderful, littl...|(390,[3,4,6,7,8,9...|
|I thought this wa...| positive|          1.0|[thought, this, w...|[thought, was, wo...|(390,[0,3,4,7,8,9...|
|Basically there's...| negative|          0.0|[basically, there...|[basically, there...|(390,[1,2,3,5,6,8...|
|Petter Mattei's "...| positive|          1.0|[petter, mattei, ...|[petter, mattei, ...|(390,[1,2,3,4,5,7...|
|Probably my all-t...| positive|          1.0|[probably, all, t...|[probably, all, t...|(390,[1,2,4,7,8,1...|
|I sure wo

In [ ]:
(trainDF,testDF) = df_vect.randomSplit((0.9,0.1),seed=42)

In [ ]:
from pyspark.ml.classification import RandomForestClassifier
# Train a RandomForest model.
rf = RandomForestClassifier(labelCol="sentiment_enc", featuresCol="rawFeatures", numTrees=10)
df_yhat= rf.fit(trainDF).transform(trainDF)

In [ ]:
df_yhat.show()

+--------------------+---------+-------------+--------------------+--------------------+--------------------+--------------------+--------------------+----------+
|              review|sentiment|sentiment_enc|              tokens|     filtered_tokens|         rawFeatures|       rawPrediction|         probability|prediction|
+--------------------+---------+-------------+--------------------+--------------------+--------------------+--------------------+--------------------+----------+
|\b\b\b\bA Turkish...| positive|          1.0|[turkish, bath, s...|[turkish, bath, s...|(390,[0,1,2,3,4,5...|[4.96418616765674...|[0.49641861676567...|       1.0|
|!!!! MILD SPOILER...| negative|          0.0|[mild, spoilers, ...|[mild, spoilers, ...|(390,[1,2,3,4,6,7...|[4.13816108671745...|[0.41381610867174...|       1.0|
|!!!! MILD SPOILER...| negative|          0.0|[mild, spoilers, ...|[mild, spoilers, ...|(390,[0,1,2,3,4,5...|[5.88587928037525...|[0.58858792803752...|       0.0|
|!!!! POSSIBLE MIL...|

In [ ]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
evaluator = MulticlassClassificationEvaluator(labelCol='sentiment_enc',predictionCol='prediction',metricName='accuracy')
accuracy = evaluator.evaluate(df_yhat)

In [ ]:
accuracy

0.7462653437215601

In [ ]:
# Lo mismo pero para el test
df_yhat_test= rf.fit(trainDF).transform(testDF)
evaluator = MulticlassClassificationEvaluator(labelCol='sentiment_enc',predictionCol='prediction',metricName='accuracy')
evaluator.evaluate(df_yhat_test)

0.743988684582744

In [ ]:
# TODO
from pyspark.mllib.evaluation import MulticlassMetrics
from pyspark.sql.functions import *
metrics = MulticlassMetrics(df_yhat_test.select(col("sentiment_enc"),col("prediction")))

#Pipelines

In [ ]:
# We will use the pipeline to automate the process of machine learning from the process of feature engineering to model building.
from pyspark.ml import Pipeline  
# Ingeniería de características
from pyspark.ml.feature import RegexTokenizer, StopWordsRemover, CountVectorizer, IDF
from pyspark.ml.classification import RandomForestClassifier

# Dejamos solo palabras
regexTokenizer = RegexTokenizer(inputCol="review", outputCol="tokens", pattern="\\W", toLowercase = True, minTokenLength = 3)
stopwords_remover = StopWordsRemover(inputCol='tokens',outputCol='filtered_tokens',stopWords=['this','these','that','those','the','and'])
# Realizamos un conteo de las palabras en cada review
vectorizer = CountVectorizer(inputCol='filtered_tokens',outputCol='rawFeatures',maxDF=.8,minDF=.05,binary=True)
# Train a RandomForest model.
rf = RandomForestClassifier(labelCol="sentiment_enc", featuresCol="rawFeatures", numTrees=10)

pipeline = Pipeline(stages=[regexTokenizer,stopwords_remover,vectorizer, rf])

In [ ]:
(trainDF,testDF) = indexed.randomSplit((0.9,0.1),seed=42)

In [ ]:
pipefit = pipeline.fit(trainDF)

In [ ]:
yhat_train = pipefit.transform(trainDF)

In [ ]:
yhat_test = pipefit.transform(testDF)

In [ ]:
yhat_test.show()

+--------------------+---------+-------------+--------------------+--------------------+--------------------+--------------------+--------------------+----------+
|              review|sentiment|sentiment_enc|              tokens|     filtered_tokens|         rawFeatures|       rawPrediction|         probability|prediction|
+--------------------+---------+-------------+--------------------+--------------------+--------------------+--------------------+--------------------+----------+
|" While sporadica...| negative|          0.0|[while, sporadica...|[while, sporadica...|(391,[0,3,5,6,8,1...|[5.49088004510498...|[0.54908800451049...|       0.0|
|"2001: A Space Od...| positive|          1.0|[2001, space, ody...|[2001, space, ody...|(391,[0,2,3,4,5,6...|[7.02264914942802...|[0.70226491494280...|       0.0|
|"Autumn Spring" t...| positive|          1.0|[autumn, spring, ...|[autumn, spring, ...|(391,[2,3,4,5,6,9...|[4.58128353069765...|[0.45812835306976...|       1.0|
|"Bela Lugosi reve...|

# Feature Selection

In [ ]:
from pyspark.ml.feature import ChiSqSelector
selector=ChiSqSelector(numTopFeatures = 200, featuresCol='rawFeatures', outputCol='selectedFeatures', labelCol= 'sentiment_enc')
model=selector.fit(yhat_train)

In [ ]:
df_features_train = model.transform(yhat_train)

In [ ]:
df_features_test = model.transform(yhat_test)

In [ ]:
# Train a RandomForest model.
rf = RandomForestClassifier(labelCol="sentiment_enc", featuresCol="selectedFeatures", numTrees=10)
rf_fit = rf.fit(df_features_train.select('selectedFeatures','sentiment_enc'))

In [ ]:
yhat_test_feat = rf_fit.transform(df_features_test.select('selectedFeatures','sentiment_enc'))

In [ ]:
yhat_test_feat.show()

+--------------------+-------------+--------------------+--------------------+----------+
|    selectedFeatures|sentiment_enc|       rawPrediction|         probability|prediction|
+--------------------+-------------+--------------------+--------------------+----------+
|(200,[0,3,6,7,8,9...|          0.0|[5.32930528748670...|[0.53293052874867...|       0.0|
|(200,[0,2,3,4,5,6...|          1.0|[6.51130083531375...|[0.65113008353137...|       0.0|
|(200,[2,3,5,6,8,2...|          1.0|[4.23837074142084...|[0.42383707414208...|       1.0|
|(200,[1,5,7,9,14,...|          0.0|[4.23837074142084...|[0.42383707414208...|       1.0|
|(200,[0,1,2,4,5,6...|          1.0|[6.03952647272759...|[0.60395264727275...|       0.0|
|(200,[1,2,4,5,8,9...|          1.0|[5.62525093445323...|[0.56252509344532...|       0.0|
|(200,[1,9,10,17,2...|          0.0|[4.52855757601603...|[0.45285575760160...|       1.0|
|(200,[0,1,2,3,4,5...|          1.0|[3.51730666604509...|[0.35173066660450...|       1.0|
|(200,[0,2

In [ ]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
evaluator = MulticlassClassificationEvaluator(labelCol='sentiment_enc',predictionCol='prediction',metricName='accuracy')
evaluator.evaluate(yhat_test_feat)

0.7395433420893109

In [ ]:
# Make new prediction
from pyspark.sql.types import StringType
ex1 = spark.createDataFrame([("'The movie was so interesting the best so far'",StringType())],["review"])
pred_ex1 = pipefit.transform(ex1)

In [ ]:
pred_ex1.select('probability').show(truncate=False)

+---------------------------------------+
|probability                            |
+---------------------------------------+
|[0.4396831634688273,0.5603168365311726]|
+---------------------------------------+

